In [17]:
import pandas as pd

# Load your dataset
df = pd.read_csv("daily_mov.csv", parse_dates=["date"])

# Sort by user and date for proper time-series operations
df.sort_values(by=["user_id", "date"], inplace=True)

# Extract 'day' and 'month' from 'date'
df["day"] = df["date"].dt.day
df["month"] = df["date"].dt.month

# Yesterday's mood
df["mood_yesterday"] = df.groupby("user_id")["mood"].shift(1)

# Average mood over last 5 days
df["mood_avg_5d"] = (
    df.groupby("user_id")["mood"]
    .rolling(window=5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

# Mood change from yesterday to today
df["mood_change"] = df["mood"] - df["mood_yesterday"]

# One-hot encode the 'weekday' column
weekday_dummies = pd.get_dummies(df["weekday"], prefix="weekday")
df = pd.concat([df, weekday_dummies], axis=1)

# Drop columns: id, date, total_entries
df.drop(columns=["id", "date", "total_entries"], inplace=True)

# Save the modified dataset
df.to_csv("daily_mov_featured.csv", index=False)

print("✅ Done! Features updated and saved to 'daily_mov_featured.csv'")


✅ Done! Features updated and saved to 'daily_mov_featured.csv'


In [18]:
# Evaluation loop
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

model.eval()
test_loss = 0
test_batches = 0
with torch.no_grad():
    all_preds = []
    all_true = []
    for batch in test_loader:
        x = batch['features'].to(device)
        user_id = batch['user_id'].to(device)
        mood = batch['mood'].to(device).unsqueeze(1)

        pred = model(x, user_id)

        if torch.isnan(pred).any():
            print(f"⚠️ NaN detected in test predictions. Batch skipped.")
            continue

        loss = loss_fn(pred, mood)
        test_loss += loss.item()
        test_batches += 1

        all_preds.extend(pred.cpu().numpy())
        all_true.extend(mood.cpu().numpy())

avg_test_loss = test_loss / max(test_batches, 1)
all_preds = np.array(all_preds).flatten()
all_true = np.array(all_true).flatten()

# Compute regression R^2 and classification accuracy (optional binning)
r2 = r2_score(all_true, all_preds)

# Discretize for rough accuracy: round mood to nearest integer 1-10
rounded_preds = np.clip(np.round(all_preds), 1, 10)
rounded_true = np.clip(np.round(all_true), 1, 10)
acc = accuracy_score(rounded_true, rounded_preds)

print(f"✅ Final Test Loss (MSE): {avg_test_loss:.4f}")
print(f"📈 R^2 Score: {r2:.4f}")
print(f"🎯 Mood Prediction Accuracy (rounded): {acc:.4f}")


⚠️ NaN detected in test predictions. Batch skipped.
⚠️ NaN detected in test predictions. Batch skipped.
⚠️ NaN detected in test predictions. Batch skipped.
⚠️ NaN detected in test predictions. Batch skipped.
✅ Final Test Loss (MSE): 0.0491
📈 R^2 Score: -0.0241
🎯 Mood Prediction Accuracy (rounded): 0.9925
